In [1]:
from datasets import load_dataset, load_metric, DatasetDict

d:\projects\postag-thai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle

# Read dictionary pkl file
with open('tag_id.pkl', 'rb') as fp:
    tag_id = pickle.load(fp)
    print(tag_id)

{'PR': 1, 'PS': 2, 'NU': 3, 'AV': 4, 'XX': 5, 'FX': 6, 'AJ': 7, 'NN': 8, 'AX': 9, 'PU': 10, 'IJ': 11, 'VV': 12, 'CC': 13, 'CL': 14, 'PA': 15, 'NG': 16}


In [3]:
pos_tag_thai = load_dataset("json", data_files="thai10_dataset_fixed.json")

pos_tag_thai['train'].features

{'id': Value(dtype='int64', id=None),
 'pos_tag': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'token': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [16]:
from datasets import ClassLabel

features = pos_tag_thai['train'].features.copy()
features["pos_tag"] = ClassLabel(names=['PR', 'PS', 'NU', 'AV', 'XX', 'FX', 'AJ', 'NN', 'AX', 'PU', 'IJ', 'VV', 'CC', 'CL', 'PA', 'NG'])
# def adjust_labels(batch):
#     batch["pos_tag"] = [sentiment + 1 for sentiment in batch["pos_tag"]]
#     return batch
# dataset = pos_tag_thai.map(adjust_labels, batched=True, features=features)

In [17]:
features

{'id': Value(dtype='int64', id=None),
 'pos_tag': ClassLabel(names=['PR', 'PS', 'NU', 'AV', 'XX', 'FX', 'AJ', 'NN', 'AX', 'PU', 'IJ', 'VV', 'CC', 'CL', 'PA', 'NG'], id=None),
 'token': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [63]:
from datasets import ClassLabel,Sequence

pos_tag_thai['train'].cast_column("pos_tag", Sequence(ClassLabel(names=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16])))

Casting the dataset:   0%|          | 0/130560 [00:00<?, ? examples/s]


ValueError: Invalid string class label NN

In [18]:
pos_tag_thai['train'].features

{'id': Value(dtype='int64', id=None),
 'pos_tag': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'token': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [27]:
train_test_pos_tag = pos_tag_thai['train'].train_test_split(test_size=0.1)
#valid_pos = train_test_pos_tag['test'].train_test_split(test_size=0.5)

pos_tag_thai = DatasetDict({
    'train': train_test_pos_tag['train'],
    #'test': valid_pos['train'],
    'valid': train_test_pos_tag['test']
})

In [28]:
pos_tag_thai

DatasetDict({
    train: Dataset({
        features: ['id', 'pos_tag', 'token'],
        num_rows: 94003
    })
    valid: Dataset({
        features: ['id', 'pos_tag', 'token'],
        num_rows: 10445
    })
})

In [6]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-th-cased")

In [7]:
tokenizer("ภาษา","ไทย","ง่าย","นิดเดียว")

{'input_ids': [11, 166, 2930, 8349, 12, 188, 5518, 3552, 12], 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [11, 142, 5724, 3552, 12, 159, 7662, 8484, 6910, 3552, 4953, 12]}

In [8]:
example = pos_tag_thai["train"][4]
print(example["token"])

['คดี', 'นี้', 'ผู้', 'ร้อง', 'มี', 'นาย', 'แก้วสรร', 'อติโพธิ', 'นาย', 'สัก', 'กอแสงเรือง']


In [9]:
tokenized_input = tokenizer(example["token"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'ค', '##ดี', 'น', '##ี', '##้', 'ผ', '##ู', '##้', 'ร', '##้อง', 'ม', '##ี', 'น', '##าย', 'แ', '##ก', '##้', '##ว', '##ส', '##ร', '##ร', 'อ', '##ติ', '##โ', '##พ', '##ธ', '##ิ', 'น', '##าย', 'ส', '##ัก', 'ก', '##อ', '##แ', '##ส', '##ง', '##เ', '##ร', '##ือ', '##ง', '[SEP]']


In [10]:
len(example[f"pos_tag"]), len(tokenized_input["input_ids"])

(11, 42)

In [11]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, None]


In [12]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"pos_tag"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

42 42


In [13]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"pos_tag"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenized_datasets = pos_tag_thai.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 13056/13056 [00:02<00:00, 5272.28 examples/s]


In [15]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("Geotrend/bert-base-th-cased", num_labels=16)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Geotrend/bert-base-th-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model_checkpoint = "Geotrend/bert-base-th-cased"
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-pos_thai",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [19]:
metric = load_metric("seqeval")

In [20]:
label_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

labels = [label_list[i] for i in example[f"pos_tag"]]
metric.compute(predictions=[labels], references=[labels])

d:\projects\postag-thai\.venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
d:\projects\postag-thai\.venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
d:\projects\postag-thai\.venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
d:\projects\postag-thai\.venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 13 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 '_': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [21]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)